In [128]:
# load package
# string 
import re

# math
import pandas as pd
import numpy as np
import scipy as sp
import random

# sys
import sys
import os
import time
import warnings

# machine learning
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, gaussian_process, discriminant_analysis
from xgboost import XGBClassifier

# model utils
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection 
from sklearn import model_selection
from sklearn import metrics

# plot
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix #??
# = show plots in Jupyter Notebook browser
%matplotlib inline 
mpl.style.use('ggplot') #??
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8 #??

# self define
sys.path.append('../../utils/')

In [6]:
# read files
lb_wsp_2014 = pd.read_csv('../data/Autohaus_weeber/leonberg_werkstattposten_2014.csv', sep = ';')
lb_wsp_2015 = pd.read_csv('../data/Autohaus_weeber/leonberg_werkstattposten_2015.csv', sep = ';')
lb_wsp_2016 = pd.read_csv('../data/Autohaus_weeber/leonberg_werkstattposten_2016.csv', sep = ';')
lb_wsp_2017 = pd.read_csv('../data/Autohaus_weeber/leonberg_werkstattposten_2017.csv', sep = ';')
lb_wsp_2018 = pd.read_csv('../data/Autohaus_weeber/leonberg_werkstattposten_2018.csv', sep = ';')
std_wsp_2014 = pd.read_csv('../data/Autohaus_weeber/weil_der_stadt_werkstattposten_2014.csv', sep = ';')
std_wsp_2015 = pd.read_csv('../data/Autohaus_weeber/weil_der_stadt_werkstattposten_2015.csv', sep = ';')
std_wsp_2016 = pd.read_csv('../data/Autohaus_weeber/weil_der_stadt_werkstattposten_2016.csv', sep = ';')
std_wsp_2017 = pd.read_csv('../data/Autohaus_weeber/weil_der_stadt_werkstattposten_2017.csv', sep = ';')
std_wsp_2018 = pd.read_csv('../data/Autohaus_weeber/weil_der_stadt_werkstattposten_2018.csv', sep = ';')

/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
# cat
d1 = lb_wsp_2014.copy()
d2 = lb_wsp_2015.copy()
d3 = lb_wsp_2016.copy()
d4 = lb_wsp_2017.copy()
d5 = lb_wsp_2018.copy()
data = pd.concat([d1,d2,d3,d4], 0)

In [87]:
d6 = std_wsp_2014.copy()
d7 = std_wsp_2015.copy()
d8 = std_wsp_2016.copy()
d9 = std_wsp_2017.copy()
d10 = std_wsp_2018.copy()
data2 = pd.concat([d6,d7,d8,d9,d10], 0)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1001054 entries, 0 to 279792
Data columns (total 21 columns):
Auftragsnummer          1001054 non-null object
AW-Nr                   437185 non-null object
Teile-Nr                517935 non-null object
KM-Stand                1000958 non-null object
Beschreibung            1000315 non-null object
Auftragsdatum           1001054 non-null object
Markencode              995567 non-null object
Lagerortcode            1001054 non-null object
Adressanredecode        991514 non-null object
Fahrgestellnummer       996850 non-null object
Motorcode               979644 non-null object
Fahrzeugmodellnummer    995486 non-null object
Modell                  993023 non-null object
Typ                     720265 non-null object
Getriebecode            976875 non-null object
Getriebeartcode         575 non-null object
Gewicht                 995390 non-null object
Leistung (KW)           995390 non-null float64
Erstzulassungsdatum     995390 non-null

In [88]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2192193 entries, 0 to 291459
Data columns (total 21 columns):
Auftragsnummer          object
AW-Nr                   object
Teile-Nr                object
KM-Stand                object
Beschreibung            object
Auftragsdatum           object
Markencode              object
Lagerortcode            object
Adressanredecode        object
Fahrgestellnummer       object
Motorcode               object
Fahrzeugmodellnummer    object
Modell                  object
Typ                     object
Getriebecode            object
Getriebeartcode         object
Gewicht                 object
Leistung (KW)           float64
Erstzulassungsdatum     object
Neuwagen                object
Unnamed: 20             float64
dtypes: float64(2), object(19)
memory usage: 368.0+ MB


In [12]:
data.isnull().sum()

Auftragsnummer                0
AW-Nr                    563869
Teile-Nr                 483119
KM-Stand                     96
Beschreibung                739
Auftragsdatum                 0
Markencode                 5487
Lagerortcode                  0
Adressanredecode           9540
Fahrgestellnummer          4204
Motorcode                 21410
Fahrzeugmodellnummer       5568
Modell                     8031
Typ                      280789
Getriebecode              24179
Getriebeartcode         1000479
Gewicht                    5664
Leistung (KW)              5664
Erstzulassungsdatum        5664
Neuwagen                 429342
Unnamed: 20             1001054
dtype: int64

## AW-Nr

In [92]:
aw = data['AW-Nr']

In [93]:
aw.isnull().sum()

563869

In [94]:
aw.describe()

count       437185
unique       18746
top       99999994
freq         15014
Name: AW-Nr, dtype: object

- 指定的是预计工时， 原来应该是float类型的，估计是有错乱
- 一半的数值是缺失的，
- 最大值top有问题啊.
- count和unique的占比是40比1， 
- 因为是数值类型的，而且值域比较大，感觉应该先进行分段
- 分布不均，觉得没有意义的数值，在里边的占比确实最大的，约1/3，

In [124]:
a = data.loc[aw == 51017133.0]
a.count()

Auftragsnummer          358
AW-Nr                   358
Teile-Nr                  0
KM-Stand                358
Beschreibung            358
Auftragsdatum           358
Markencode              355
Lagerortcode            358
Adressanredecode        344
Fahrgestellnummer       357
Motorcode               347
Fahrzeugmodellnummer    355
Modell                  351
Typ                     236
Getriebecode            343
Getriebeartcode           0
Gewicht                 355
Leistung (KW)           355
Erstzulassungsdatum     355
Neuwagen                174
Unnamed: 20               0
dtype: int64

In [118]:
aw.unique()

array([nan, 51017133.0, 51017195.0, ..., '13731905', '58656500', '66341900'], dtype=object)

## Teile-Nr

In [98]:
tn = data['Teile-Nr']

In [99]:
tn.isnull().sum()

483119

In [100]:
tn.describe() # 相比于14年的有增加

count          517935
unique          37008
top       N  90813202
freq            13922
Name: Teile-Nr, dtype: object

- 指部件的号码，每个号码指不同的部件
- 一半的值是空值。数量和独立的比是20比1，也就是说5年间，他们所处理的任务极少是重复的。
- 因为这是编号：应该具体了解一下编号各个数值的定义，从而合并一些小的范畴，减少数量，最简单的方法是每个数值作为一种新的属性，进行分析。
- 分布不均 N. 占了1/5

## KM-Stand

In [18]:
ka = data['KM-Stand']

In [25]:
ka.isnull().sum()

96

In [19]:
ka.describe() # 相比于14年的有

count     1000958
unique      52607
top         15,00
freq        11404
Name: KM-Stand, dtype: object

- 指里程表， 应该是数值类型的，但是这里却不是，所以应该出现了一些乱码，可以用正则表达式找一下
- 只有少量的null值，单看是可以忽略掉，数值类型可分段
- 分布不均， 有1万多个是1500的

## Beschreibung

In [20]:
bs = data['Beschreibung']

In [26]:
bs.isnull().sum()

739

In [21]:
bs.describe() # 相比于14年的有

count      1000315
unique       61567
top       SCHRAUBE
freq         86845
Name: Beschreibung, dtype: object

- 对维护的描述
- 有少量的空值
- 类型多，每种类型大概出现有14次，可能是大小写导致的独立数目偏多？？？
- 分布不均， schraube自己就占了8万多个

In [127]:
b = data.loc[bs == 'Karosserie Lackierung vorbereitet']
b

,Auftragsnummer,AW-Nr,Teile-Nr,KM-Stand,Beschreibung,Auftragsdatum,Markencode,Lagerortcode,Adressanredecode,Fahrgestellnummer,...,Fahrzeugmodellnummer,Modell,Typ,Getriebecode,Getriebeartcode,Gewicht,Leistung (KW),Erstzulassungsdatum,Neuwagen,Unnamed: 20
347,WSAU214922,NaN,NaN,"64311,00",Karosserie Lackierung vorbereitet,30.06.2014,SKODA,"Weeber Leonberg Skoda (Hertichstraße 25, 71229...",Firma,TMBKT61Z4C2187428,...,1Z522Y,OCTAVIA COMBI II Ambiente 4X4,NaN,NFQ,NaN,"1705,00",77.0,21.06.2012,als Neuwagen bezogen,NaN
409,WSAU214922,NaN,NaN,"64311,00",Karosserie Lackierung vorbereitet,30.06.2014,SKODA,"Weeber Leonberg Skoda (Hertichstraße 25, 71229...",Firma,TMBKT61Z4C2187428,...,1Z522Y,OCTAVIA COMBI II Ambiente 4X4,NaN,NFQ,NaN,"1705,00",77.0,21.06.2012,als Neuwagen bezogen,NaN
411,WSAU214922,NaN,NaN,"64311,00",Karosserie Lackierung vorbereitet,30.06.2014,SKODA,"Weeber Leonberg Skoda (Hertichstraße 25, 71229...",Firma,TMBKT61Z4C2187428,...,1Z522Y,OCTAVIA COMBI II Ambiente 4X4,NaN,NFQ,NaN,"1705,00",77.0,21.06.2012,als Neuwagen bezogen,NaN
412,WSAU214922,NaN,NaN,"64311,00",Karosserie Lackierung vorbereitet,30.06.2014,SKODA,"Weeber Leonberg Skoda (Hertichstraße 25, 71229...",Firma,TMBKT61Z4C2187428,...,1Z522Y,OCTAVIA COMBI II Ambiente 4X4,NaN,NFQ,NaN,"1705,00",77.0,21.06.2012,als Neuwagen bezogen,NaN
420,WSAU214922,NaN,NaN,"64311,00",Karosserie Lackierung vorbereitet,30.06.2014,SKODA,"Weeber Leonberg Skoda (Hertichstraße 25, 71229...",Firma,TMBKT61Z4C2187428,...,1Z522Y,OCTAVIA COMBI II Ambiente 4X4,NaN,NFQ,NaN,"1705,00",77.0,21.06.2012,als Neuwagen bezogen,NaN
421,WSAU214922,NaN,NaN,"64311,00",Karosserie Lackierung vorbereitet,30.06.2014,SKODA,"Weeber Leonberg Skoda (Hertichstraße 25, 71229...",Firma,TMBKT61Z4C2187428,...,1Z522Y,OCTAVIA COMBI II Ambiente 4X4,NaN,NFQ,NaN,"1705,00",77.0,21.06.2012,als Neuwagen bezogen,NaN
1748,WSAU200473,NaN,NaN,"933,00",Karosserie Lackierung vorbereitet,07.01.2014,AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",Firma,WAUZZZ4H1EN010108,...,4HC0TA,Audi A8 Limousine 4.2 TDI clea,X0A,PRX,NaN,"0,00",0.0,11.12.2013,als Neuwagen bezogen,NaN
1749,WSAU200473,NaN,NaN,"933,00",Karosserie Lackierung vorbereitet,07.01.2014,AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",Firma,WAUZZZ4H1EN010108,...,4HC0TA,Audi A8 Limousine 4.2 TDI clea,X0A,PRX,NaN,"0,00",0.0,11.12.2013,als Neuwagen bezogen,NaN
1750,WSAU200473,NaN,NaN,"933,00",Karosserie Lackierung vorbereitet,07.01.2014,AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",Firma,WAUZZZ4H1EN010108,...,4HC0TA,Audi A8 Limousine 4.2 TDI clea,X0A,PRX,NaN,"0,00",0.0,11.12.2013,als Neuwagen bezogen,NaN
1853,WSAU200480,NaN,NaN,"18409,00",Karosserie Lackierung vorbereitet,02.01.2014,AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",Frau,WAUZZZ8R4BA043108,...,8RB0MY,Audi Q5 SUV 2.0 TDI quattro,X0A,MSJ,NaN,"0,00",0.0,07.07.2011,als Neuwagen bezogen,NaN


## Auftragsdatum

In [28]:
ad = data['Auftragsdatum']

In [29]:
ad.isnull().sum()

0

In [30]:
ad.describe()

count        1001054
unique          1257
top       14.11.2016
freq            2036
Name: Auftragsdatum, dtype: object

- 因为是时间，所以会想知道时间跨度，每个事件段里出现的次数。还有数量改变的大概趋势

## Markencode

In [31]:
mc = data['Markencode']

In [32]:
mc.isnull().sum()

5487

In [33]:
mc.describe()

count     995567
unique        32
top           VW
freq      552930
Name: Markencode, dtype: object

- 汽车品牌
- 有少量缺失值
- 分布不均，VW占了超过一半的量，会想看分布图

## Lagerortcode

In [34]:
lc = data['Lagerortcode']

In [35]:
lc.isnull().sum()

0

In [36]:
lc.describe()

count                                               1001054
unique                                                    4
top       Weeber Leonberg VW (Glemseckstraße 49, 71229 L...
freq                                                 570972
Name: Lagerortcode, dtype: object

- 车间地址
- 没有空值，只有4个车间
- 分布不均，其中一个车间占了一般的量

## Adressanredecode

In [37]:
ac = data['Adressanredecode']

In [38]:
ac.isnull().sum()

9540

In [39]:
ac.describe()

count     991514
unique        14
top        Firma
freq      459305
Name: Adressanredecode, dtype: object

- 指客户类型，
- 少量缺失值
- 仅14类，分布不均匀，大部分客户是来自公司的，约占总体的一半

In [83]:
ac.unique()

array(['Firma', 'Herr', 'Frau', nan, 'AN', 'FA_F', 'FA_M', 'HR.DR.',
       'OHNE', 'AUTOHAUS', 'FR.DR.', 'HERR+FRAU', 'FAHRSCHULE', 'FR.PROF.',
       'HR U. FR'], dtype=object)

## Fahrgestellnummer

In [40]:
fn = data['Fahrgestellnummer']

In [41]:
fn.isnull().sum()

4204

In [42]:
fn.describe()

count                996850
unique                24584
top       WVWZZZAUZFW144310
freq                    846
Name: Fahrgestellnummer, dtype: object

- 汽车底盘上的编号，不知道是否唯一的，如果是就可以用来识别车辆，急需确认。。。。。。。。。。， 估计不是，因为最大的重复量是800多，应该不可能有人会在5年间重复维修汽车那么多次
- 少量的空值，分布相对比较均匀了

## Motorcode


In [43]:
mc = data['Motorcode']

In [44]:
mc.isnull().sum()

21410

In [45]:
mc.describe()

count     979644
unique       908
top         CFFB
freq       61955
Name: Motorcode, dtype: object

## Fahrzeugmodellnummer

In [46]:
fn = data['Fahrzeugmodellnummer']

In [47]:
fn.isnull().sum()

5568

In [48]:
fn.describe()

count     995486
unique      4444
top       36535Y
freq       11110
Name: Fahrzeugmodellnummer, dtype: object

## Modell

In [49]:
md = data['Modell']

In [50]:
md.isnull().sum()

8031

In [51]:
md.describe()

count                             993023
unique                              4796
top       Passat Variant Comfortline BM 
freq                               19857
Name: Modell, dtype: object

- 对比单号，可能是同一个人的所以模型是一样的，

## Typ

In [52]:
tp = data['Typ']

In [53]:
tp.isnull().sum()

280789

In [54]:
tp.describe()

count     720265
unique        95
top          X0A
freq      635469
Name: Typ, dtype: object

- 还不知道是什么意思
- 大量的缺失值
- 基本都是X0A，分布极端不均匀

In [82]:
tp.unique()

array(['X0A', nan, 'X10', 'Golf', 'Fabia', 'A4', 'Superb Combi', 'Eos',
       'Transporter', '1K', 'Octavia', 'Polo', '1B15C1', 'Phaeton',
       '3c5056', 'Roomster', 'Octavia Combi', '104919', 'Golf Plus',
       '3c21h2', 'Tiguan', 'Touran', 'California', '3C5056', '2kb cz 2',
       'A6', '1k10e1', 'A5', '3T527Y', '1K21L3', 'TDI Basis', 'A4 Avant',
       'Fabia Combi', '1Z5345', '5J', '^', 'A3', '1k1BT2', '36535X',
       'Octavia TDI', '1J', '7HA162', '1K51L2', 'Ocatvia',
       'Passat Variant', '9N30M7', 'Touran 1.4 TSI', 'Caddy', 'Focus',
       '3C5052', 'Style Edition', '1k10g2', 'x10', '5l7', ' (6R1) BlueGT',
       '163', 'Audi A4', 'Fox', '9N12F4', 'Audi A6 Allroad quattro',
       'Audi Q5', 'Caddy Maxi K0 Trend 77', 'Cayenne', 'Passat', 'Golf V',
       '2CB T19', '1,4 HybHigh 110', '6C1BNX', '518I', 'A3 Cabrio', 'Leon',
       'D 1.2 Twinport', '7M8285', 'S-MAx', '5422b4',
       'BlueMotion Technology 2,0l TDI', 'A3 Sportback', '9N', 'A 8 L',
       '6R12B4', 'Coupe'

## Getriebecode

In [58]:
gc = data['Getriebecode']

In [59]:
gc.isnull().sum()

24179

In [60]:
gc.describe()

count     976875
unique      2124
top          NFU
freq       16295
Name: Getriebecode, dtype: object

## Getriebeartcode

In [61]:
gac = data['Getriebeartcode']

In [62]:
gac.isnull().sum()

1000479

In [63]:
gac.describe()

count     575
unique     14
top       M38
freq      385
Name: Getriebeartcode, dtype: object

## Gewicht

In [64]:
gw = data['Gewicht']

In [65]:
gw.isnull().sum()

5664

In [66]:
gw.describe()

count     995390
unique        40
top         0,00
freq      978648
Name: Gewicht, dtype: object

- 无数个零

## Leistung (KW)


In [69]:
ls = data['Leistung (KW)']

In [70]:
ls.isnull().sum()

5664

In [71]:
ls.describe()

count    995390.000000
mean         35.580716
std          56.387796
min           0.000000
25%           0.000000
50%           0.000000
75%          75.000000
max        1258.000000
Name: Leistung (KW), dtype: float64

- 大量为0

## Erstzulassungsdatum

In [72]:
ed = data['Erstzulassungsdatum']

In [73]:
ed.isnull().sum()

5664

In [74]:
ed.describe()

count         995390
unique          4723
top       01.01.1753
freq            6396
Name: Erstzulassungsdatum, dtype: object

- 不知道指的是什么

## Neuwagen

In [75]:
nw = data['Neuwagen']

In [76]:
nw.isnull().sum()

429342

In [77]:
nw.describe()

count                   571712
unique                       1
top       als Neuwagen bezogen
freq                    571712
Name: Neuwagen, dtype: object

- 估计没写的就不是新车